IMT 2200 - Introducción a Ciencia de Datos<br>
**Pontificia Universidad Católica de Chile**<br>
**Instituto de Ingeniería Matemática y Computacional**<br>
**Semestre 2025-S2**<br>
**Profesor:** Rodrigo A. Carrasco <br>

---

## Proyecto de Investigación – Terremotos en Chile


## Pregunta 5:
## **Modelo predictivo de lugares de riesgo**<br>

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from os import chdir, getcwd
if 'Preguntas'in getcwd():
    chdir('../')

In [4]:
df = pd.read_csv('data//query_limpio.csv', index_col=0)
df.columns

Index(['level_0', 'index', 'latitude', 'longitude', 'depth', 'mag', 'magType',
       'nst', 'gap', 'dmin', 'rms', 'id', 'horizontalError', 'depthError',
       'magError', 'magNst', 'date', 'hour', 'zone', 'region'],
      dtype='object')

In [19]:
x = df[['depth', 'mag', 'magType', 'nst', 'gap',
    'dmin', 'rms', 'horizontalError', 'depthError',
    'magError', 'magNst', 'date', 'hour']]
y = df['zone']

In [ ]:
cat_cols = x.select_dtypes(include=["object"]).columns
num_cols = x.select_dtypes(exclude=["object"]).columns

preprocessor = ColumnTransformer(
    transformers=[
        ("categorical", 
        Pipeline([
            ("imputer", SimpleImputer(strategy="most_frequent")),
            ("onehot", OneHotEncoder(handle_unknown="ignore"))
        ]),
        cat_cols),
        
        ("numerical",
        Pipeline([
            ("imputer", SimpleImputer(strategy="median"))
        ]),
        num_cols)
    ]
)

In [23]:
model = RandomForestClassifier(n_estimators=100, random_state=42)

clf = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", model)
])

X_train, X_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=42
)

In [24]:
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.27783964365256125
                      precision    recall  f1-score   support

               Ancud       0.00      0.00      0.00         5
               Angol       0.00      0.00      0.00         4
         Antofagasta       0.15      0.31      0.20        64
              Arauco       1.00      0.06      0.11        17
               Arica       0.00      0.00      0.00        23
                Buin       0.00      0.00      0.00         1
              Bulnes       0.00      0.00      0.00         4
              Calama       0.37      0.81      0.51       162
             Calbuco       0.00      0.00      0.00         1
              Camiña       0.00      0.00      0.00         5
             Carahue       0.00      0.00      0.00        22
           Cartagena       0.11      0.06      0.08        17
              Castro       0.00      0.00      0.00         1
           Cauquenes       0.00      0.00      0.00        12
              Cañete       0.14      0.

C:\Users\josem\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\josem\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
C:\Users\josem\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} i